Implementation of [Rodriguez](https://ieeexplore.ieee.org/document/5475260) (sec. III)

In [1]:
# imports & constants
import numpy as np
import matplotlib.pyplot as plt
from math import floor, ceil

SEED = 0
np.random.seed(SEED)

In [2]:
# generate single tone
amplitude = 1
nominal_frequency = 50
phi = np.pi/4
f_s = 44_100 # 1_000
duration = 3

n = np.arange(f_s * duration)
s_tone = amplitude*np.sin(2 * np.pi * nominal_frequency / f_s * n + np.radians(phi))
del n

In [3]:
# A. Phase Estimation Using the DFT
M = len(s_tone)
hann_window = np.hanning(M)
x = s_tone * hann_window
N_DFT = 2*M
X = np.fft.fft(x, n=N_DFT)
# freqs = np.fft.fftfreq(NDFT, 1/f_s)
# f_DFT = freqs[k_peak]

k_peak = np.argmax(np.abs(X[:N_DFT//2]))
f_DFT = k_peak*f_s/N_DFT
print(f"f_DFT: {f_DFT:.2f}")

phi_DFT = np.angle(X[k_peak]) # % (2*np.pi)
print(f"phi_DFT: {phi_DFT:.2f}")

f_DFT: 50.00
phi_DFT: -1.56


In [4]:
# B. The Novel Phase Estimation Method
# 1
s_dash = f_s * np.diff(s_tone, prepend=0)

# 2
M = len(s_tone)
hann_window = np.hanning(M)
x = s_tone * hann_window
x_dash = s_dash * hann_window

# 3
N_DFT = 2*M+1
X = np.fft.fft(x, n=N_DFT)
X_dash = np.fft.fft(x_dash, n=N_DFT)

# 4
abs_X = np.abs(X)
abs_X_dash = np.abs(X_dash)
k_peak = np.argmax(abs_X)
print(f"k_peak: {k_peak}")

# 5
F_k_peak = (np.pi * k_peak) / (N_DFT * np.sin(np.pi * k_peak / N_DFT))

DFT_0_peak = abs_X[k_peak]
DFT_1_peak = F_k_peak * abs_X_dash[k_peak]

# 6
f_DFT1 = 1 / (2 * np.pi) * DFT_1_peak / DFT_0_peak
k_DFT1 = N_DFT * f_DFT1 / f_s
print(f"f_DFT1: {f_DFT1:.2f}")
print(f"k_DFT1: {k_DFT1}")

# validating
if not -.5 <= k_DFT1 - k_peak < .5:
    raise ValueError("invalid result")

k_peak: 300
f_DFT1: 50.00
k_DFT1: 300.0011338450338


In [5]:
# continuing with the method

omega_0 = 2*np.pi*f_DFT1/f_s
k_low = floor(k_DFT1)
k_high = ceil(k_DFT1)
theta_low = np.angle(X_dash[k_low])
theta_high = np.angle(X_dash[k_high])

theta = (k_DFT1 - k_low) * (theta_high - theta_low) / (k_high - k_low) + theta_low
# theta = np.interp(k_DFT1, [k_low, k_high], [theta_low, theta_high])

numerator = np.tan(theta) * (1 - np.cos(omega_0)) + np.sin(omega_0)
denominator = 1 - np.cos(omega_0) - np.tan(theta) * np.sin(omega_0)
phi_DFT1 = np.arctan(numerator / denominator)
# phi_DFT1 = np.arctan2(numerator, denominator)

# choosing phi_DFT1 closer to phi_DFT
_sym_intvl = lambda x: (x + np.pi) % (2*np.pi) - np.pi
_phi_DFT1 = _sym_intvl(phi_DFT1 + np.pi)
if np.abs(_phi_DFT1 - phi_DFT) < np.abs(phi_DFT1- phi_DFT):
    phi_DFT1 = _phi_DFT1
    
print(f"phi_DFT1: {phi_DFT1:.2f}")

phi_DFT1: -1.56
